# param recovery

In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
import scipy.stats as stats
import seaborn as sns
from sklearn.metrics import confusion_matrix
# np.random.seed(42)
from joblib import Parallel, delayed
import matplotlib.tri as tri
import matplotlib.colors as mcolors
from scipy.interpolate import griddata
from scipy.interpolate import RBFInterpolator
import matplotlib.ticker as mticker
import itertools
from sklearn.metrics import r2_score


In [2]:
output_dir = "27_RL_agent_TDlearn_output_both_param_recovery"
os.makedirs(output_dir, exist_ok=True)


folder_path_participants = 'data_risk_added_epileptic'
folder_path_colors_numbers = '13_RL_agent_TDlearn_output_risk_sensitive/model_behavior'


df_participants = []
df_colors_numbers = []


def find_matching_csv(folder_path, df_list):
            for csv_file in os.listdir(folder_path):
                if clean_name in csv_file and csv_file.endswith('.csv'):
                    csv_path = os.path.join(folder_path, csv_file)
                    df_csv = pd.read_csv(csv_path)
                    df_list.append(df_csv)





for file_name in os.listdir(folder_path_participants):
    if file_name.endswith('.csv'):
        file_path = os.path.join(folder_path_participants, file_name)
        df = pd.read_csv(file_path)
        df = df[df['outcome'].str.lower() != 'na'].reset_index(drop=True) 
        df_participants.append(df)

        clean_name = file_name.removeprefix("task_data_").removesuffix(".csv")
        find_matching_csv(folder_path_colors_numbers, df_colors_numbers)


In [3]:
for df in df_participants:
    df['block_type'] = None

    df.loc[df['block'] == 1, 'block_type'] = 'uniform'     # Block 1 is uni
    df.loc[df['block'] == 4, 'block_type'] = 'mix'     # Block 4 is mix

    # For blocks 2 and 3, set based on distribution
    df.loc[(df['block'] == 2) & (df['distribution'] == 'low'), 'block_type'] = 'low'
    df.loc[(df['block'] == 2) & (df['distribution'] == 'high'), 'block_type'] = 'high'
    df.loc[(df['block'] == 3) & (df['distribution'] == 'low'), 'block_type'] = 'low'
    df.loc[(df['block'] == 3) & (df['distribution'] == 'high'), 'block_type'] = 'high'
    



for i in range(len(df_participants)):
    myCard = df_participants[i]['myCard']
    yourCard = df_participants[i]['yourCard']
    distributions = df_participants[i]['distribution']
    block_type = df_participants[i]['block_type']
    
    for df_list in [ df_colors_numbers]:
        df_list[i]['myCard'] = myCard
        df_list[i]['yourCard'] = yourCard
        df_list[i]['distribution'] = distributions
        df_list[i]['block_type'] = block_type

In [4]:
for df in df_colors_numbers:
    df['model_choices'] = df['model_choices'].replace({1: 'arrowup', 0: 'arrowdown'})

In [5]:
for df in df_colors_numbers:
    outcomes = []
    for i in range(len(df)):
        my = df.loc[i, 'myCard']
        your = df.loc[i, 'yourCard']
        choice = df.loc[i, 'model_choices']
        
        if ((my > your and choice == "arrowup") or (my < your and choice == "arrowdown")):
            outcomes.append('win')
        else:
            outcomes.append('lose')
    
    df['outcome'] = outcomes

In [6]:
participants = [os.path.splitext(file)[0].replace("task_data_", "")
    for file in os.listdir(folder_path_participants) if file.endswith('.csv')]

In [7]:
df_combined = pd.concat(df_colors_numbers, ignore_index=True)

df_combined = df_combined[df_combined['outcome'].str.lower() != 'na'].reset_index(drop=True)
 

desired_order = ["uniform", "low", "high"]  


cards_sorted = sorted(df_combined["myCard"].unique())
dist_sorted = [d for d in desired_order if d in df_combined["distribution"].unique()]
choice_sorted = sorted(df_combined["model_choices"].unique())


card_idx = {card: i for i, card in enumerate(cards_sorted)}
dist_idx = {dist: i for i, dist in enumerate(dist_sorted)}
choice_idx = {choice: i for i, choice in enumerate(choice_sorted)}


matrix_3d = np.zeros((len(cards_sorted), len(dist_sorted), len(choice_sorted)))


for _, row in df_combined.iterrows():
    i = card_idx[row["myCard"]]
    j = dist_idx[row["distribution"]]
    k = choice_idx[row["model_choices"]]
    matrix_3d[i, j, k] += 1  


total_per_card_dist = matrix_3d.sum(axis=2, keepdims=True)

# compute percentages, avoiding division by zero
with np.errstate(divide='ignore', invalid='ignore'):
    percentage_matrix = np.divide(matrix_3d, total_per_card_dist, where=total_per_card_dist != 0)

# convert to a DataFrame for easy visualization
percentage_list = []
for i, card in enumerate(cards_sorted):
    for j, dist in enumerate(dist_sorted):
        for k, choice in enumerate(choice_sorted):
            percentage_list.append({
                "myCard": card,
                "distribution": dist,  # Now follows "uniform", "low", "high" order
                "model_choices": choice,
                "percentage": percentage_matrix[i, j, k]
            })

df_percentages = pd.DataFrame(percentage_list)
df_percentages

,myCard,distribution,model_choices,percentage
0,1,uniform,arrowdown,0.852273
1,1,uniform,arrowup,0.147727
2,1,low,arrowdown,0.803922
3,1,low,arrowup,0.196078
4,1,high,arrowdown,0.933333
5,1,high,arrowup,0.066667
6,2,uniform,arrowdown,0.969231
7,2,uniform,arrowup,0.030769
8,2,low,arrowdown,0.746032
9,2,low,arrowup,0.253968


In [8]:
actions = { "arrowdown": 0, "arrowup": 1}
distributions_map = { "uniform": 0, "low": 1,  "high": 2}
card_numbers = list(range(1, 10))

policy_table = percentage_matrix 

Q_table_init = np.random.normal(0, 0.1, (len(card_numbers), len(distributions_map), len(actions)))
# having a q-table based on the policies
# Q_table_init = policy_table * np.mean(Q_table_init) 
Q_table = Q_table_init.copy()

#############################################################################################
# having a q-table that starts with 0! this was not a good initilization so i changed it.
# Q_table = np.zeros((len(distributions_map), len(actions)))  # 3 distributions × 2 actions
#############################################################################################

print("policy: \n",np.shape(policy_table))
print("\n Q_table: \n",np.shape(Q_table))



policy: 
 (9, 3, 2)

 Q_table: 
 (9, 3, 2)


In [9]:
def softmax(Q_values, beta):
    # this part subtracts the maximum q-value in each row it means each state to improve numerical stability.
    # because exxponentials of large numbers can lead to overflow errors, so shifting q-values avoids this problem.
    
    Q_shifted = Q_values - np.max(Q_values, axis=2, keepdims=True)
    exps = np.exp(beta * Q_shifted)
    sums = np.sum(exps, axis=2, keepdims=True)
    new_probs = exps / sums

    return new_probs



def train_rescorla_wagner(df, alpha_plus, alpha_minus, beta, Q_init=None):
    if Q_init is None:
        Q_init = Q_table.copy()
    Q_values = Q_init.copy()
    q_value_pairs = []
    choices = []
    predicted_probs = []
    distributions = []
    card_numbers = []
    
    for _, row in df.iterrows():
        action = actions[row["model_choices"]] 
        distribution = distributions_map[row["distribution"]] 
        card_number = row["myCard"]-1 # since I'm using this as an index! I need to do -1 to make the 1 to 9 cards come to 0 to 8
        reward = 0.5 if row["outcome"] == "win" else -0.5


        probs = softmax(Q_values, beta)
        predicted_probs.append(probs[card_number][distribution][action])
        
        prediction_error = reward - Q_values[card_number][distribution][action]
        if prediction_error > 0:    # prediction is less than the real reward
            alpha = alpha_plus
        else:                       # prediction is more than the real reward
            alpha = alpha_minus
               
        Q_values[card_number][distribution][action] += alpha * prediction_error
        
        q_value_pairs.append(Q_values.copy())
        choices.append(action)
        distributions.append(distribution)
        card_numbers.append(card_number)
        

    return np.array(q_value_pairs), np.array(choices), np.array(predicted_probs), np.array(distributions), np.array(card_numbers)


# this is for the sake of parallel computing
def compute_log_likelihood(alpha_plus, alpha_minus, beta, df_all, Q_table):
    Q_init_participant = Q_table.copy()
    q_values, choices, predicted_probs, distributions, card_numbers = train_rescorla_wagner(df_all, alpha_plus, alpha_minus, beta, Q_init=Q_init_participant.copy())
    
    predicted_probs = np.clip(predicted_probs, 1e-6, 1)  # prevent log(0)
    log_likelihood = np.sum(np.log(predicted_probs))
    
    return (alpha_plus, alpha_minus, beta, log_likelihood)


In [10]:
num_of_samples = 100
# num_of_samples = 1000
alpha_min = 0.01
alpha_max = 1
beta_min = 0.01
beta_max  = 8
alpha_plus_samples = np.random.uniform(alpha_min, alpha_max + np.finfo(float).eps, num_of_samples)
alpha_minus_samples = np.random.uniform(alpha_min, alpha_max + np.finfo(float).eps, num_of_samples)
beta_samples = np.random.uniform(beta_min, beta_max + np.finfo(float).eps, num_of_samples)

In [11]:
df_colors_numbers[0]

,model_choices,participant_choices,model_total_reward,participant_total_reward,q_val,myCard,yourCard,distribution,block_type,outcome
0,arrowdown,0,10.5,10.5,"[[[-0.029411727012342598, 0.02214272258434538]...",2,7,uniform,uniform,win
1,arrowup,1,11.0,11.0,"[[[-0.029411727012342598, 0.02214272258434538]...",9,4,uniform,uniform,win
2,arrowdown,0,11.5,11.5,"[[[-0.029411727012342598, 0.02214272258434538]...",5,6,uniform,uniform,win
3,arrowup,1,12.0,12.0,"[[[-0.029411727012342598, 0.02214272258434538]...",9,1,uniform,uniform,win
4,arrowdown,1,11.5,12.5,"[[[-0.029411727012342598, 0.02214272258434538]...",6,4,uniform,uniform,lose
...,...,...,...,...,...,...,...,...,...,...
265,arrowup,1,83.0,85.0,"[[[0.4999843904309233, 0.02214272258434538], [...",8,7,high,mix,win
266,arrowdown,0,83.5,85.5,"[[[0.4999843904309233, 0.02214272258434538], [...",4,8,high,mix,win
267,arrowdown,1,84.0,85.0,"[[[0.4999843904309233, 0.02214272258434538], [...",8,9,low,mix,win
268,arrowdown,0,83.5,84.5,"[[[0.4999843904309233, 0.02214272258434538], [...",3,1,low,mix,lose


In [12]:
BIC_models = []
AIC_models = []
best_alpha_plus_models = []
best_alpha_minus_models = []
best_beta_models = []
accuracy_models = []
precision_models = []
sensitivity_recall_models = []
specificity_models = []
f1_score_models = []
mcFadden_r2_models = []
r2_models = []

for idx, df_all in enumerate(df_colors_numbers):
    print(f"Processing participant {idx + 1} of {len(df_colors_numbers)}")
    Q_init_participant = Q_table.copy()

    df_all = df_all[df_all['outcome'].str.lower() != 'na'].reset_index(drop=True)

    results = Parallel(n_jobs=-1, backend='loky')(
        delayed(compute_log_likelihood)(alpha_plus, alpha_minus, beta, df_all, Q_init_participant.copy())
        for alpha_plus in alpha_plus_samples
        for alpha_minus in alpha_minus_samples
        for beta in beta_samples
    )

    alpha_beta_log_likelihood = {}
    best_log_likelihood = -np.inf
    best_alpha_plus = -1
    best_alpha_minus = -1
    best_beta = -1

    for alpha_plus, alpha_minus, beta, log_likelihood in results:
        alpha_beta_log_likelihood[(alpha_plus, alpha_minus, beta)] = log_likelihood
        if log_likelihood > best_log_likelihood:
            best_log_likelihood = log_likelihood
            best_alpha_plus, best_alpha_minus, best_beta = alpha_plus, alpha_minus, beta

    results_df = pd.DataFrame(alpha_beta_log_likelihood.keys(), columns=["alpha_plus", "alpha_minus", "beta"])
    results_df["log_likelihood"] = alpha_beta_log_likelihood.values()

    
    q_values, choices, predicted_probs, distributions, card_numbers = train_rescorla_wagner(df_all, best_alpha_plus, best_alpha_minus, best_beta, Q_init=Q_init_participant.copy())
    
    
    predicted_choices = []
    for trial in range(len(card_numbers)):
        test_action_probs = softmax(q_values[trial], best_beta)
        p_arrowup = test_action_probs[card_numbers[trial]][distributions[trial]][actions["arrowup"]]
        p_arrow_down = test_action_probs[card_numbers[trial]][distributions[trial]][actions["arrowdown"]]
        # choosing 1 or 0 based on the softmax probabilities:
        predicted_choices.append(np.random.choice([1, 0], p=[p_arrowup, p_arrow_down]))


    # finding out model total reward based on the model's predicted choices
    total_reward = [] 
    for i in range(len(predicted_choices)):
        if len(total_reward)> 0:
            last_reward = total_reward[-1]  #  the last reward value
        else:
            last_reward = 10 # initial reward is $10
        
        if ((df_all.loc[i, 'myCard'] > df_all.loc[i, 'yourCard'] and predicted_choices[i] == 1) or
            (df_all.loc[i, 'myCard'] < df_all.loc[i, 'yourCard'] and predicted_choices[i] == 0)):
            total_reward.append(last_reward + 0.5)
        else:
            total_reward.append(last_reward - 0.5)

    
    
    # confusion matrix:
    conf_matrix = confusion_matrix(choices, predicted_choices)
    TN, FP, FN, TP = conf_matrix.ravel()  # unpacking the confusion matrix
    # acc
    accuracy = (TP + TN) / (TP + TN + FP + FN)
    # precision: From the ones that we’ve announced them as up/down, which ones are really up/down?
    precision = TP / (TP + FP) if (TP + FP) != 0 else 0
    # recall or sensitivity : true positive rate
    sensitivity_recall = TP / (TP + FN) if (TP + FN) != 0 else 0
    # specificity : true negative rate
    specificity = TN / (TN + FP) if (TN + FP) != 0 else 0
    # f1 Score
    f1_score = 2 * (precision * sensitivity_recall) / (precision + sensitivity_recall) if (precision + sensitivity_recall) != 0 else 0

    
    # bayes information criterion:
    n_trials = len(df_all)
    k = 3  # number of free parameters: alpha and beta
    BIC = k * np.log(n_trials) - 2 * best_log_likelihood # this is BIC formula based on the log lkelihode I found before

        # Akaike  information criterion(AIC):
    AIC = 2 * k - 2 * best_log_likelihood 

    # mcFadden r-squared:
    p_null = np.mean(choices)  # probability of choosing "1" in the dataset
    log_likelihood_null = np.sum(choices * np.log(p_null) + (1 - choices) * np.log(1 - p_null))
    mcFadden_r2 = 1 - (best_log_likelihood / log_likelihood_null)

    # r-squared
    r2 = r2_score(choices, predicted_choices)
    
    
    # saving models evaluation variables:
    best_alpha_plus_models.append(best_alpha_plus)
    best_alpha_minus_models.append(best_alpha_minus)
    best_beta_models.append(best_beta)
    BIC_models.append(BIC)
    AIC_models.append(AIC)
    accuracy_models.append(accuracy)
    precision_models.append(precision)
    sensitivity_recall_models.append(sensitivity_recall)
    specificity_models.append(specificity)
    f1_score_models.append(f1_score)
    mcFadden_r2_models.append(mcFadden_r2)
    r2_models.append(r2)


Processing participant 1 of 8
Processing participant 2 of 8
Processing participant 3 of 8
Processing participant 4 of 8
Processing participant 5 of 8
Processing participant 6 of 8
Processing participant 7 of 8
Processing participant 8 of 8


# now saving the model evaluation values

In [13]:
df_models_evaluation = pd.DataFrame({
    "participants": participants,
    "best_alpha_plus": best_alpha_plus_models,
    "best_alpha_minus": best_alpha_minus_models,
    "best_beta": best_beta_models,
    "BIC": BIC_models,
    "AIC": AIC_models,
    "accuracy": accuracy_models,
    "precision": precision_models,
    "sensitivity_recall": sensitivity_recall_models,
    "specificity": specificity_models,
    "f1_score": f1_score_models,
    "mcFadden_r2": mcFadden_r2_models,
    "r2": r2_models
})

file_path = os.path.join(output_dir, "models_evaluation_risk_sensitive.csv")
df_models_evaluation.to_csv(file_path, index=False)